In [112]:
import numpy as np
import pandas as pd

from ast import literal_eval
import os, time

# Random Functions

In [113]:
RESULT_DIR = "final_results"
benchmark = "rational_sqrt"

In [114]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
    except Exception as e:
        pass

In [115]:
df_res = pd.DataFrame({
    # "Function": df['faza']['formula'],
})
for tool in sorted(df):
    df_res[tool.capitalize()+"_output"] = df[tool]['output'].fillna(df[tool]['error']).astype(str).str[:500]
    if tool == "volesti":
        df['volesti']['volumes'] = df['volesti']['details'].apply(literal_eval).apply(lambda x: np.asarray([t['output'] for t in x]))
        a = df['volesti']['volumes'].apply(lambda x: x.mean())
        b = df['volesti']['volumes'].apply(lambda x: x.std())
        df_res[tool.capitalize()+"_meanstd"] = list(np.vstack((a, b)).T)
    df_res[tool.capitalize()+"_time"] = df[tool]['time'].apply(lambda x: round(x, 2))

df_res.head() 

,Gubpi_output,Gubpi_time,Psi_output,Psi_time
0,"(0.5850834106982069, 0.5853559563800296)",0.43,E[p_] = 1/Integrate[1/(-226/625+1/xi1^2*580436...,1.92
1,"(0.6940753026396893, 0.6997831992174012)",0.52,E[p_] = 1/(-1/(100/99*Integrate[Boole[-1+xi1<=...,0.04
2,"(0.983213423102785, 0.9838691427004288)",0.53,E[p_] = (20/89)^(1/2)*451869997/100000000,0.02
3,"(0.7722084144661071, 0.7725548915902053)",0.42,E[p_] = -30155^(1/2)*646/54945+646/54945*85100...,0.04
4,"(1.2874941229630317, 1.2888450045581636)",0.58,E[p_] = 1/(-1/(100/99*Integrate[Boole[-1+xi1<=...,0.04


In [116]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)